In [1]:
from os.path import abspath
from os.path import dirname as up
import numpy as np
import sys

In [2]:
# Insert path to pybf library to system path
path_to_lib ='/home/vsergei/Projects/repos'
print(path_to_lib)
sys.path.insert(0, path_to_lib)

/home/vsergei/Projects/repos


In [3]:
from pybf.pybf.io_interfaces import DataLoader
from pybf.scripts.beamformer_cartesian_realtime import BFCartesianRealTime
from pybf.pybf.image_settings import ImageSettings
from pybf.pybf.visualization import plot_image

/home/sem21f11/miniconda3/envs/pybf_env/lib/python3.6/importlib/__init__.py:126: MatplotlibDeprecationWarning: 
The matplotlib.backends.backend_qt4agg backend was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  return _bootstrap._gcd_import(name[level:], package, level)


In [4]:
dataset_path = "/home/vsergei/Courses/UFIMA 2020/Final project/Picmus/database/experiments/resolution_distorsion/rf_dataset.hdf5"

In [5]:
data_loader_obj = DataLoader(dataset_path)

### Image settings

In [6]:
### Specify Image settings and create corresponding object ###

img_res = [400, 600]
image_x_range = [-0.019, 0.019]
image_z_range = [0.005, 0.05]

db_range = 50

LATERAL_PIXEL_DENSITY_DEFAULT = 5

img_config = ImageSettings(image_x_range[0],
                           image_x_range[1],
                           image_z_range[0],
                           image_z_range[1],
                           LATERAL_PIXEL_DENSITY_DEFAULT,
                           data_loader_obj.transducer)

### Specify preprocessing parameters for RF data ###

decimation_factor = 1
interpolation_factor = 10

### Specify TX strategy and Apodization parameters ###

start_time = 0
correction_time_shift = 0

alpha_fov_apod = 40

# 1 Plane waves with inclination angle 0
# tx_strategy = ['PW_75_16', [16]]
tx_strategy = ['PW_1_0', [0]]

### Specify Sampling Frequency ###

SAMPLING_FREQ = 20.832 * (10 ** 6)

filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

The highest resolution for the system is:  (633, 205)


In [7]:
tx_strategy = ['PW_1_0', [0]]

### Instantiate beamformer for single Plane Wave with 0 inclination angle

In [8]:
bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                         tx_strategy,
                         data_loader_obj.transducer,
                         decimation_factor,
                         interpolation_factor,
                         img_res,
                         img_config,
                         start_time=start_time,
                         correction_time_shift=correction_time_shift,
                         alpha_fov_apod=alpha_fov_apod,
                         bp_filter_params=filters_params,
                         envelope_detector='hilbert',
                         picmus_dataset=True)

Delays precalculation...
TX strategy: plane waves
Number of plane waves:  1
Maximum angle:  0.0 °
Apodization precalculation...


### Beamform

In [9]:
rf_data = data_loader_obj.get_rf_data(0, 37)

In [10]:
img_data = bf.beamform(rf_data, numba_active=True)

Beamforming...
 
Time of execution: 2.7455239295959473 seconds


### Visualize PW 0

In [11]:
_ = plot_image(np.abs(img_data), 
               scatters_coords_xz=None,
               elements_coords_xz=None,
               framework='plotly',
               title='Original Image (PW_0)',
               image_x_range=image_x_range,
               image_z_range=image_z_range,
               db_range=db_range,
               colorscale='Greys',
               save_fig=True, 
               show=False,
               path_to_save='.')

BF Final dB range (-50.0,0.0)


### Instatiate beamformer for Plane waves 75 with max angle 16

In [12]:
print('Transmit Strategy: ')
print(data_loader_obj.tx_strategy[0])

Transmit Strategy: 
PW_75_16


In [13]:
bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                         data_loader_obj.tx_strategy,
                         data_loader_obj.transducer,
                         decimation_factor,
                         interpolation_factor,
                         img_res,
                         img_config,
                         start_time=start_time,
                         correction_time_shift=correction_time_shift,
                         alpha_fov_apod=alpha_fov_apod,
                         bp_filter_params=filters_params,
                         envelope_detector='hilbert',
                         picmus_dataset=True)

Delays precalculation...
TX strategy: plane waves
Number of plane waves:  75
Maximum angle:  16.0 °
Apodization precalculation...


### Beamform

#### Retrieve the data

In [14]:
rf_data_shape = (data_loader_obj.num_of_acq_per_frame,) + data_loader_obj.get_rf_data(0, 0).shape
print('RF data shape: ', rf_data_shape)

rf_data = np.zeros(rf_data_shape)
for i in range(rf_data.shape[0]):
    rf_data[i, :, :] = data_loader_obj.get_rf_data(0, i)

RF data shape:  (75, 128, 3328)


### Launch beamforming

In [15]:
img_data = bf.beamform(rf_data, numba_active=True)

Beamforming...
 
Time of execution: 46.60521101951599 seconds


### Visualize PW 75 image

In [16]:
_ = plot_image(np.abs(img_data), 
               scatters_coords_xz=None,
               elements_coords_xz=None,
               framework='plotly',
               title='Original Image (PW_75)',
               image_x_range=image_x_range,
               image_z_range=image_z_range,
               db_range=db_range,
               colorscale='Greys',
               save_fig=True, 
               show=True,
               path_to_save='.')

BF Final dB range (-50.0,0.0)
